<a href="https://colab.research.google.com/github/ft10101/AI-ML/blob/main/Day6_Ensemble_Bagging(diabetes_prediction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align='center'>Ensemble Learning: Bagging Tutorial</h2>

**We will use pima indian diabetes dataset to predict if a person has a diabetes or not based on certain features such as blood pressure, skin thickness, age etc. We will train a standalone model first and then use bagging ensemble technique to check how it can improve the performance of the model**

dataset credit: https://www.kaggle.com/gargmanas/pima-indians-diabetes

In [12]:
import pandas as pd
df = pd.read_csv("/content/pima-indians-diabetes.csv")
df

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
762,10,101,76,48,180,32.9,0.171,63,0
763,2,122,70,27,0,36.8,0.340,27,0
764,5,121,72,23,112,26.2,0.245,30,0
765,1,126,60,0,0,30.1,0.349,47,1


In [4]:
df.describe()

,6,148,72,35,0,33.6,0.627,50,1
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.842243,120.859192,69.101695,20.517601,79.903520,31.990482,0.471674,33.219035,0.348110
std,3.370877,31.978468,19.368155,15.954059,115.283105,7.889091,0.331497,11.752296,0.476682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,32.000000,0.371000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,127.500000,36.600000,0.625000,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
new_row = {'6': 'Pregnancies', '148': 'Glucose', '72': 'BloodPressure', '35':'SkinThickness','0':'Insulin','33.6':'BMI', '0.627':'DiabetesPedigreeFunction','50':'Age','1':'Outcome'}

In [10]:
df.iloc[0] = new_row

In [9]:
df.head()

,6,148,72,35,0,33.6,0.627,50,1
0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [14]:
# create a dictionary with the new column names
new_columns = {'6': 'Pregnancies', '148': 'Glucose', '72': 'BloodPressure', '35': 'SkinThickness', 
               '0': 'Insulin', '33.6': 'BMI', '0.627': 'DiabetesPedigreeFunction', '50': 'Age', '1': 'Outcome'}

# use the rename method to change the column names
df = df.rename(columns=new_columns)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [17]:
df.isnull().sum() #number of all that are null

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [16]:
df.Outcome.value_counts() 

0    500
1    267
Name: Outcome, dtype: int64

<h3>Train test split</h3>

In [18]:
X = df.drop("Outcome",axis="columns")
y = df.Outcome

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:3]

array([[-0.84372629, -1.12208597, -0.16024856,  0.53202348, -0.69355921,
        -0.68372895, -0.36426474, -0.18894038],
       [ 1.23423997,  1.94447577, -0.26357823, -1.28688187, -0.69355921,
        -1.10230105,  0.60470064, -0.1037951 ],
       [-0.84372629, -0.99692019, -0.16024856,  0.15569823,  0.12235685,
        -0.49346891, -0.91968415, -1.0403932 ]])

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=10)

In [21]:
X_train.shape

(575, 8)

In [22]:
X_test.shape

(192, 8)

In [23]:
y_train.value_counts()

0    375
1    200
Name: Outcome, dtype: int64

In [24]:
201/375

0.536

In [25]:
y_test.value_counts()

0    125
1     67
Name: Outcome, dtype: int64

In [26]:
67/125

0.536

<h3>Train using stand alone model</h3>

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

array([0.70779221, 0.67532468, 0.70588235, 0.79084967, 0.73202614])

In [28]:
scores.mean()

0.7223750106103047

<h3>Train using Bagging</h3>

In [29]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(X_train, y_train)
bag_model.oob_score_

0.7634782608695653

In [30]:
bag_model.score(X_test, y_test)

0.7552083333333334

In [31]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores

array([0.75324675, 0.73376623, 0.75816993, 0.81699346, 0.73202614])

In [32]:
scores.mean()

0.7588405058993294

We can see some improvement in test score with bagging classifier as compared to a standalone classifier

<h3>Train using Random Forest</h3>

In [33]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), X, y, cv=5)
scores.mean()

0.771912401324166